In [1]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import polars as pl
import nfl_data_py as nfl

env = "local"

In [2]:
if env == "local":
    os.chdir("/Users/samuel/Documents/GitHub/QB-GPT/")
else:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/MyDrive/NFL_Challenge/NFL-GPT/NFL data")

In [3]:
os.listdir()

['data_models',
 '.DS_Store',
 'app',
 'LICENSE',
 'models',
 'README.md',
 '.gitignore',
 '.gitattributes',
 'indexv2',
 'data_preprocessing',
 'index',
 '.git',
 'notebooks']

In [4]:
training_data = tf.data.Dataset.load("data_models/StratFormer/train_data")
testing_data = tf.data.Dataset.load("data_models/StratFormer/test_data")

In [5]:
train_length = [i for i,_ in enumerate(training_data)][-1] + 1
test_length = [i for i,_ in enumerate(testing_data)][-1] + 1

In [6]:
print("Train length is : ", str(train_length))
print("Test length is : ", str(test_length))

Train length is :  941860
Test length is :  403655


In [7]:
batch_size = 32

training_data = training_data.shuffle(train_length).batch(batch_size)
testing_data = testing_data.shuffle(test_length).batch(batch_size)

In [8]:
from models.modeling.QBGPT.models import QBGPT, LargeQBGPT

moves_to_pred = 11170
input_size = 11172
starts_size = 1954
scrimmage_size = 100
positions_id = 29

temp_ids = 52
off_def_size = 2
token_type_size = 3
play_type_size = 9

model_large = LargeQBGPT(input_vocab_size = input_size,
                         positional_vocab_size = temp_ids,
                         position_vocab_size=positions_id,
                         start_vocab_size=starts_size,
                         scrimmage_vocab_size=scrimmage_size,
                         offdef_vocab_size = off_def_size,
                         type_vocab_size = token_type_size,
                         playtype_vocab_size = play_type_size,
                         embedding_dim = 128,
                         hidden_dim = 128,
                         num_heads = 3,
                         diag_masks = True,
                         to_pred_size = moves_to_pred)

model_medium = QBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = off_def_size,
                    type_vocab_size = token_type_size,
                    playtype_vocab_size = play_type_size,
                    embedding_dim = 256,
                    hidden_dim = 256,
                    num_heads = 3,
                    diag_masks = True,
                    to_pred_size = moves_to_pred)

model_small = QBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = off_def_size,
                    type_vocab_size = token_type_size,
                    playtype_vocab_size = play_type_size,
                    embedding_dim = 128,
                    hidden_dim = 128,
                    num_heads = 3,
                    diag_masks = True,
                    to_pred_size = moves_to_pred)

model_tiny = QBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = off_def_size,
                    type_vocab_size = token_type_size,
                    playtype_vocab_size = play_type_size,
                    embedding_dim = 64,
                    hidden_dim = 64,
                    num_heads = 3,
                    diag_masks = True,
                    to_pred_size = moves_to_pred)

model_tiny.load_weights("models/modeling/QBGPT/weights/model_tinyv2/QBGPT")
model_small.load_weights("models/modeling/QBGPT/weights/model_smallv2/QBGPT")
model_medium.load_weights("models/modeling/QBGPT/weights/model_mediumv2/QBGPT")
model_large.load_weights("models/modeling/QBGPT/weights/model_largev2/QBGPT")

In [9]:
from models.modeling.StratFormer.models import StratFormer

In [10]:
stratformer_tiny = StratFormer(num_spec_token= 1,
                            hidden_dim=64,
                            team_vocab_size=32,
                            player_vocab_size=7229,
                            season_vocab_size= 7,
                            down_vocab_size= 5,
                            base_encoder = model_tiny.Encoder)

stratformer_tiny.Encoder.BaseEncoder.trainable = False

stratformer_small = StratFormer(num_spec_token= 1,
                            hidden_dim=128,
                            team_vocab_size=32,
                            player_vocab_size=7229,
                            season_vocab_size= 7,
                            down_vocab_size= 5,
                            base_encoder = model_small.Encoder)

stratformer_small.Encoder.BaseEncoder.trainable = False

stratformer_medium = StratFormer(num_spec_token= 1,
                            hidden_dim=256,
                            team_vocab_size=32,
                            player_vocab_size=7229,
                            season_vocab_size= 7,
                            down_vocab_size= 5,
                            base_encoder = model_medium.Encoder)

stratformer_medium.Encoder.BaseEncoder.trainable = False

stratformer_large = StratFormer(num_spec_token= 1,
                            hidden_dim=256,
                            team_vocab_size=32,
                            player_vocab_size=7229,
                            season_vocab_size= 7,
                            down_vocab_size= 5,
                            base_encoder = model_large.Encoder)

stratformer_large.Encoder.BaseEncoder.trainable = False

In [11]:
stratformer_tiny.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss= tf.keras.losses.BinaryCrossentropy(),
                          metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])

stratformer_small.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss= tf.keras.losses.BinaryCrossentropy(),
                          metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])

stratformer_medium.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss= tf.keras.losses.BinaryCrossentropy(),
                          metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])

stratformer_large.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss= tf.keras.losses.BinaryCrossentropy(),
                          metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()])

In [12]:
history_tiny = stratformer_tiny.fit(training_data, validation_data=testing_data, epochs=1)

2023-10-29 18:48:26.505849: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:12: Filling up shuffle buffer (this may take a while): 365804 of 936966
2023-10-29 18:48:36.505853: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:12: Filling up shuffle buffer (this may take a while): 729297 of 936966


   23/29281 [..............................] - ETA: 3:33 - loss: 1.1752 - binary_accuracy: 0.5842 - auc: 0.5051   

2023-10-29 18:48:42.245635: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


29277/29281 [============================>.] - ETA: 0s - loss: 0.0367 - binary_accuracy: 0.9851 - auc: 0.9986

2023-10-29 18:55:11.333141: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 363012 of 401558
2023-10-29 18:55:12.387828: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


29281/29281 [==============================] - 488s 16ms/step - loss: 0.0367 - binary_accuracy: 0.9851 - auc: 0.9986 - val_loss: 0.0026 - val_binary_accuracy: 0.9993 - val_auc: 0.9999


In [14]:
history_small = stratformer_small.fit(training_data, validation_data=testing_data, epochs=1)

2023-10-26 16:17:02.988829: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:12: Filling up shuffle buffer (this may take a while): 364871 of 736942


   16/23030 [..............................] - ETA: 4:28 - loss: 2.7159 - binary_accuracy: 0.5488 - auc_1: 0.5362    

2023-10-26 16:17:13.480495: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


23030/23030 [==============================] - 481s 20ms/step - loss: 0.1149 - binary_accuracy: 0.9584 - auc_1: 0.9912 - val_loss: 0.0159 - val_binary_accuracy: 0.9952 - val_auc_1: 0.9995


In [12]:
history_medium = stratformer_medium.fit(training_data, validation_data=testing_data, epochs=1)

2023-11-06 12:29:01.123059: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:12: Filling up shuffle buffer (this may take a while): 363513 of 941860
2023-11-06 12:29:11.123074: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:12: Filling up shuffle buffer (this may take a while): 725649 of 941860


    7/29434 [..............................] - ETA: 10:00 - loss: 12.4414 - binary_accuracy: 0.5491 - auc_2: 0.4704  

2023-11-06 12:29:17.077936: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


29433/29434 [============================>.] - ETA: 0s - loss: 0.1116 - binary_accuracy: 0.9670 - auc_2: 0.9908

2023-11-06 12:43:44.309279: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 362300 of 403655
2023-11-06 12:43:45.458464: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


29434/29434 [==============================] - 1051s 35ms/step - loss: 0.1116 - binary_accuracy: 0.9670 - auc_2: 0.9908 - val_loss: 0.0346 - val_binary_accuracy: 0.9887 - val_auc_2: 0.9985


In [16]:
history_large = stratformer_large.fit(training_data, validation_data=testing_data, epochs=1)

2023-10-26 16:37:30.897159: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:12: Filling up shuffle buffer (this may take a while): 368063 of 736942


    7/23030 [..............................] - ETA: 10:03 - loss: 8.8860 - binary_accuracy: 0.5625 - auc_3: 0.5685 

2023-10-26 16:37:41.242013: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


23030/23030 [==============================] - 867s 37ms/step - loss: 0.1658 - binary_accuracy: 0.9462 - auc_3: 0.9849 - val_loss: 0.0641 - val_binary_accuracy: 0.9809 - val_auc_3: 0.9967


In [13]:
#stratformer_tiny.Encoder.save_weights("models/modeling/StratFormer/weights/stratformer_tiny/StratFormer/")
#stratformer_small.Encoder.save_weights("models/modeling/StratFormer/weights/stratformer_small/StratFormer/")
stratformer_medium.Encoder.save_weights("models/modeling/StratFormer/weights/stratformer_medium/StratFormer/")
#stratformer_large.Encoder.save_weights("models/modeling/StratFormer/weights/stratformer_large/StratFormer/")